# MDF dataset

In [52]:
import pandas as pd
import numpy as np
import pprint
import os
from tqdm import tqdm
import holidays
from sklearn.preprocessing import MinMaxScaler
import random

pd.options.display.max_columns = 1000
pd.options.display.max_rows = 20

## Merge datasets

In [53]:
def fix_nan_category(df):
    """
    System apps like camera and gallery have NaN category, and different name for the same app (ex. samsung camera and huawei camera)
    This function fix the category and assign a common name to system apps
    """
    df.loc[df['app'].str.contains('camera'), 'category'] = 'PHOTOGRAPHY' # change category from NaN
    df.loc[df['app'].str.contains('camera'), 'app'] = 'camera'  # change app name, all camera apps from various brands are equivalent
    
    df.loc[df['app'].str.contains('com.android.incallui'), 'category'] = 'COMMUNICATION' # incallui is the interface during a call
    
    df.loc[df['app'].str.contains('mail'), 'category'] = 'PRODUCTIVITY'
    df.loc[df['app'].str.contains('com.google.android.gm'), 'category'] = 'PRODUCTIVITY' # change gmail category from communication to productivity
    
    df.loc[df['app'].str.contains('gallery'), 'category'] = 'PHOTOGRAPHY' # change category from NaN
    df.loc[df['app'].str.contains('gallery'), 'app'] = 'gallery'  # change app name, all gallery apps from various brands are equivalent

In [54]:
def filter_wifi_scans(folder_path):
    """
    Opens wifi_scans.csv file
    group by time and assign true to a group only if there is at least one row with connected == true
    skips the process if the file wifi_scans2.csv already exists
    """
    if os.path.isfile(folder_path+'/wifi_scans2.csv'):
        return
    a = pd.read_csv(folder_path+'/wifi_scans.csv')
    b = a[['time', 'connected']].groupby(['time'], as_index=False).any() # any() returns true if at least one entry is true
    b.to_csv(folder_path+'/wifi_scans2.csv', index=False)

In [55]:
def get_closest_row(file_path, columns, dt):
    """
    finds the row in a dataframe whose time column is closest to dt

    :file_path: CSV file location on disk
    :columns: columns to read when opening the file
    :dt: time in ms
    :return: closest row as numpy array
    """ 
    df = pd.read_csv(file_path, header=0, usecols=['time']+columns) # read only selected CSV columns + time column
    df['time'] = pd.to_datetime(df['time'], unit='ms') # convert from ms to date
    df.sort_values('time', inplace=True)
    df.drop_duplicates(subset='time', keep="first", inplace=True)
    df.set_index('time', inplace=True)
    closest = df.iloc[[df.index.get_loc(dt, method='nearest')]].values[0] # find nearest row to time dt
    return np.asarray(closest)

In [56]:
# dictionary structured as file : columns
file_dict = {'activities.csv': ['in_vehicle', 'on_bicycle', 'on_foot', 'running', 'still', 'tilting', 'walking', 'unknown'], 
             'audio.csv': ['ringer_mode', 'alarm_volume', 'music_volume', 'notifications_volume', 'ring_volume', 'music_active', 'speaker_on', 'headset_connected'],
             'battery.csv': ['level', 'charging'],
             'display.csv': ['state', 'rotation'],
             'weather.csv': ['temp', 'humidity', 'pressure', 'wind_speed', 'wind_deg',  'clouds', 'rain_last3h'],
             'wifi_scans2.csv': ['connected'],
             'location.csv': ['label', 'place_type']}

In [57]:
data_path = 'Datasets/MDF/'
# system apps like launcher,package manager, settings, ota...
ignored_apps = """it.cnr.iit.sensapp com.android.systemui com.sec.android.app.launcher com.android.settings com.android.vending
                  com.android.captiveportallogin com.google.android.packageinstaller com.teslacoilsw.launcher com.android.packageinstaller
                  com.samsung.android.MtpApplication com.sec.android.emergencylauncher com.wssyncmldm com.huawei.android.launcher
                  com.huawei.systemmanager com.asus.launcher android com.asus.ime com.asus.dm com.cyanogenmod.trebuchet
                  org.cyanogenmod.resolver com.android.launcher3 com.oneplus.ota com.samsung.android.game.gametools
                  com.samsung.android.app.galaxyfinder com.huawei.gamebox.global com.sec.android.inputmethod com.android.phone 
                  com.samsung.android.scloud com.huawei.android.internal.app com.miui.home com.android.providers.downloads.ui
                  com.android.printspooler com.lge.launcher3 com.lge.phonemanagement com.lge.bluetoothsetting com.lge.wifisettings
                  com.lge.homeselector com.lge.launcher2 com.lge.lockscreensettings it.cnr.iit.contextlabeler
                  com.sec.android.preloadinstaller com.android.server.telecom com.asus.powersaver com.android.stk
                  it.cnr.iit.mymoviedb """.split() 

In [58]:
for user in range(31): # foreach user folder
    user_dir = data_path + 'user_' + str(user)
    filter_wifi_scans(user_dir)

In [59]:
df = pd.DataFrame()   
for user in list(range(0,27)) + list(range(28,31)): # foreach user folder, skip user 27 it doesn't works for some reasons
    print(f"working on user {user}...")
    user_dir = data_path + 'user_' + str(user)
    
    df1 = pd.read_csv(user_dir + '/running_apps.csv', header=0) # read running apps dataframe and use it as a starting point
    df1 = df1[~df1['app'].isin(ignored_apps)]  # ignore system apps
    fix_nan_category(df1)  # fix gallery, camera...
    df1 = df1[~df1.app.str.contains("samsung|huawei|lge|asus|xiaomi|cyanogenmod")] # ignore brand apps
    df1 = df1[~df1.category.isnull()]  # ignore apps with NaN category
    df1['time'] = pd.to_datetime(df1['time'], unit='ms') # convert date from ms to datetime
    df1.sort_values('time', inplace=True)
    # df1.drop_duplicates(subset='time', keep="first", inplace=True) # drop time duplicate
    df1.reset_index(drop=True, inplace=True)
    df1.insert(1,'user',user) # insert user ID column
    
    rows = []
    for dt in tqdm(df1['time']): # foreach row in running apps dataframe find the closest row in all other dataframes using datetime
        row = []
        for filename, columns in file_dict.items(): # foreach csv file in user folder
            file_path = user_dir + '/' + filename
            row = row + get_closest_row(file_path, columns, dt).tolist() # single row with all the context features
        rows.append(row)

    df2 = pd.DataFrame(rows, columns=np.concatenate(list(file_dict.values()))) # from list of list to dataframe
    df3 = pd.concat([df1, df2], axis=1) # concat by column
    df = pd.concat([df, df3], axis=0) # concat by row
    
df.reset_index(drop=True, inplace=True)
print("done!")

  8%|██████▉                                                                            | 1/12 [00:00<00:01,  9.93it/s]

working on user 0...


  0%|                                                                                          | 0/918 [00:00<?, ?it/s]

working on user 1...


  1%|▋                                                                                 | 3/356 [00:00<00:13, 25.44it/s]

...

working on user 29...


  0%|                                                                                         | 0/2529 [00:00<?, ?it/s]

working on user 30...


100%|██████████████████████████████████████████████████████████████████████████████| 2529/2529 [04:02<00:00, 10.41it/s]

done!


In [60]:
df.to_csv('MDF_not_encoded.csv', index=False)

df = pd.read_csv('MDF_not_encoded.csv')
df['time'] = pd.to_datetime(df['time'])

## Extract new features

In [ ]:
def daytime_from_date(date):
    hour = date.hour
    if hour >= 5 and hour <= 12:
        return 'morning'
    elif hour >= 13 and hour <= 18:
        return 'afternoon'
    elif hour >= 19 and hour <= 22:
        return 'evening'
    else:
        return 'night'
    
def weekday_from_date(date):
    return date.strftime("%A")

def is_weekend(weekday:str):
    return True if weekday == 'Saturday' or weekday == 'Sunday' else False

it_holidays = holidays.Italy()

def is_holiday(date):
    return date in it_holidays

In [ ]:
df['daytime'] = df['time'].apply(daytime_from_date)
df['weekday'] = df['time'].apply(weekday_from_date)
df['is_weekend'] = df['weekday'].apply(is_weekend)
df['is_holiday'] = df['time'].apply(is_holiday)

## Encoding
### Fix labels

**place type**: group similar labels under a more general labels (es. food, restaurant and bar under food label)

In [62]:
df.loc[df['place_type'].isin(['restaurant', 'bar', 'cafe', 'food']), 'place_type'] = 'food_and_drink'
df.loc[df['place_type'].isin(['route', 'street', 'park', 'tourist_attraction']), 'place_type'] = 'outdoors'
df.loc[df['place_type'].isin(['transit_station', 'bus_station', 'taxi_stand']), 'place_type'] = 'public_transport_station'
df.loc[df['place_type'].isin(['supermarket', 'home_goods', 'bakery', 'shopping_mall', 'library', 'book_store', 'florist']), 'place_type'] = 'store'
df.loc[df['place_type'].isin(['health', 'doctor']), 'place_type'] = 'health'
df.loc[df['place_type'].isin(['finance', 'gas_station', 'general_contractor', 'bank', 'premise', 'lawyer', 'insurance_agency', 'hair_care', 'city_hall', 'plumber', 'pharmacy', 'police', 'veterinary', 'laundry', 'place_of_worship', 'university', 'moving_company', 'post_office', 'car_repair', 'real_estate_agency', 'painter', 'car_wash', 'local_government_office', 'beauty_salon', 'electrician', 'car_rental', 'funeral_home', 'fire_station', 'travel_agency']), 'place_type'] = 'service'

**category**: group all GAME subcategories under GAME label

In [63]:
df.loc[df['category'].str.contains('GAME'), 'category'] = 'GAME'
df.loc[df['category'].isin([' COMMUNICATION']), 'category'] = 'COMMUNICATION' # fix communication category with space at the beginning

### App
Convert **app** from package name to unique IDs and rename to item

In [64]:
df.app = pd.factorize(df.app)[0]
df = df.rename(columns={'app': 'item'})

### Category
**Category** is one hot encoded

In [ ]:
df = pd.concat((df, pd.get_dummies(df['category'], prefix='category')), axis=1)
df.pop('category')

### Activities
**in_vehicle, on_bicycle, on_foot, running, still, tilting, walking, unknown** represent the probability from 0 to 100 that the user is doing an activity. These features are normalized

In [ ]:
activities = 'in_vehicle on_bicycle on_foot running still tilting walking unknown'.split()
df[activities] = df[activities].apply(lambda x: x/100)

### Volume
- **ringer_mode** is one hot encoded
- **alarm_volume, music_volume, notifications_volume, ring_volume, music_active, speaker_on, headset_connected** are already normalized

In [ ]:
df = pd.concat((df, pd.get_dummies(df['ringer_mode'], prefix='ringer_mode')), axis=1)
df.pop('ringer_mode')

### Battery
- Battery **level** goes from 0 to 1, where 1 is full charged, it is converted to a categorical variable and then one-hot encoded
- **charging** is boolean

In [ ]:
def get_battery_status(lv):
    lv = lv * 100
    if lv >= 80:
        return 'charged'
    elif lv >= 60 and lv < 80:
        return 'quite charged'
    elif lv >= 40 and lv < 60:
        return 'half charged'
    elif lv >= 20 and lv < 40:
        return 'low'
    else:
        return 'very low'

df['level'] = df['level'].apply(get_battery_status)
df = pd.concat((df, pd.get_dummies(df['level'], prefix='battery')), axis=1)
df.pop('level')

### Display
- **state** can be 1,2,3,4
- **rotation** can be 0,1,3

Both variables are one hot encoded

In [ ]:
df = pd.concat((df, pd.get_dummies(df['state'], prefix='display_state')), axis=1)
df = pd.concat((df, pd.get_dummies(df['rotation'], prefix='display_rotation')), axis=1)
df.pop('state')
df.pop('rotation')

### Weather
- **temp**, **humidity, pressure, wind_speed, wind_deg**, **clouds** are normalized
- **rain_last3h** is transformed into a boolean

In [ ]:
df['rain_last3h'] = df['rain_last3h'].apply(lambda x: 1 if x > 0 else 0) # true if it rained

cols_to_norm = 'temp humidity pressure wind_speed wind_deg clouds'.split()
df[cols_to_norm] = MinMaxScaler().fit_transform(df[cols_to_norm])

### Place and date
**place_type, daytime, weekday** are one hot encoded

In [ ]:
cols = 'place_type daytime weekday'.split()
for e in cols:
    df = pd.concat((df, pd.get_dummies(df[e], prefix=e)), axis=1)
    df.pop(e)

### Boolean to int

In [ ]:
for col in 'music_active speaker_on headset_connected connected is_weekend is_holiday'.split():
    x[col] = x[col].astype(int)

### Add rating

In [65]:
df.pop('time')
df['rating'] = 1

## Negative sampling

In [66]:
neg_df = pd.DataFrame(columns=df.columns)

all_labels = df.label.unique() # all possible context of a single user

items_labels = {} # dictionary that contains in which contexts an item has been used
for item in df.item.unique():
    items_labels[item] = df[df.item == item]['label'].unique()
    
for index, row in df.iterrows():
    item = row['item']
    pos_labels = items_labels[item]  # contexts in which an item has been used
    neg_labels = list(set(all_labels) - set(pos_labels))  # contexts in which an item has NOT been used
    for neg in neg_labels: # generate a new negative sample foreach negative label
        neg_context = df.loc[(df.item != item) & (df.label == neg)].sample(n=1) # take a random item with negative context
        neg_context = neg_context.iloc[:, 4:] # keep only the context
        item_row = pd.DataFrame(row.iloc[0:4]).transpose() # take user, item, rating
        item_row.reset_index(drop=True, inplace=True) # reset index for concat
        neg_context.reset_index(drop=True, inplace=True)
        neg_row = pd.concat([item_row, neg_context], axis=1)
        neg_row.rating = 0
        neg_df = neg_df.append(neg_row)   

df = df.append(neg_df)
df.sort_values(by=['user'], inplace=True)
df.reset_index(drop=True, inplace=True)
# df.pop('label')

In [67]:
df.to_csv('MDF_not_encoded_neg_sampled.csv')

## Save to CSV

In [ ]:
df.to_csv('MDF_final.csv', index=False)